# Analyse topological divergences and the baselines for Lyapunov estimation


In [24]:
import pandas as pd
import os
import pickle

RESULTS_DIR = "outputs/data/divergence_results"
pkl_files = [
    f
    for f in os.listdir(RESULTS_DIR)
    if f.endswith(("_500.pkl", "_750.pkl", "_1000.pkl"))
]

data_frames = []

for pkl_file in pkl_files:
    # Load the dictionary from the .pkl file
    with open(f"{RESULTS_DIR}/{pkl_file}", "rb") as file:
        data_dict = pickle.load(file)

    # Convert dictionary into a Pandas DataFrame
    data_frame = pd.DataFrame(list(data_dict.items()), columns=["Keys", "Values"])

    # Split the tuple keys into separate columns
    data_frame[["Measure", "System", "Length"]] = pd.DataFrame(
        data_frame["Keys"].tolist(), index=data_frame.index
    )

    # Drop the original keys column
    data_frame = data_frame.drop(["Keys"], axis=1)

    # Use the new columns as a multi-index
    # data_frame.set_index(["measure", "system", "length"], inplace=True)

    # Add the dataframe to the list
    data_frames.append(data_frame)

# Concatenate the data frames
merged_data = pd.concat(data_frames)

# Split values dictionaries into separate columns and drop original column
merged_data = pd.concat(
    [merged_data.drop(["Values"], axis=1), merged_data["Values"].apply(pd.Series)],
    axis=1,
)

# Further split the two correlation columns into the coefficient and pvalues
merged_data[["Spearman Rho (All)", "Spearman p (All)"]] = merged_data[
    "spearmanr"
].apply(pd.Series)
merged_data[["Spearman Rho (Chaos)", "Spearman p (Chaos)"]] = merged_data[
    "pos_spearmanr"
].apply(pd.Series)

# Forget the original columns that were split
merged_data = merged_data.drop(columns=["spearmanr", "pos_spearmanr"])

# Forget useless divergence
# ignore_measures = (
#     "interleaving_length_0.5_None divergence",
#     "interleaving_length_0.4_None divergence",
#     "interleaving_length_0.3_None divergence",
#     "interleaving_length_0.2_None divergence",
#     "interleaving_length_0.1_None divergence",
#     "cophenetic_length",
#     "cophenetic_reverse_length",
# )
# merged_data = merged_data[~merged_data["Measure"].isin(ignore_measures)]

# Ensure the system names are consistent across records
merged_data["System"] = merged_data["System"].replace("Hénon", "Henon")
merged_data["System"] = merged_data["System"].replace("henon", "Henon")
merged_data["System"] = merged_data["System"].replace("ikeda", "Ikeda")
merged_data["System"] = merged_data["System"].replace("logistic", "Logistic")
merged_data["System"] = merged_data["System"].replace("tinkerbell", "Tinkerbell")

# Create readable column names
merged_data.rename(
    columns={
        "classification_f1": "Random Forest F1 Score",
        "regression_neg_mean_absolute": "Random Forest MSE (All)",
        "pos_regression_neg_mean_absolute": "Random Forest MSE (Chaos)",
        "regression_neg_mean_absolute_poly": "kNN MSE (All)",
        "pos_regression_neg_mean_absolute_poly": "kNN MSE (Chaos)",        
    },
    inplace=True
)


In [25]:
merged_data.head()

,Measure,System,Length,Random Forest F1 Score,Random Forest MSE (All),Random Forest MSE (Chaos),kNN MSE (All),kNN MSE (Chaos),Spearman Rho (All),Spearman p (All),Spearman Rho (Chaos),Spearman p (Chaos)
0,cophenetic divergence,Henon,500,0.770,-0.088,-0.065,-0.085,-0.060,0.736,0.000,0.747,0.000
1,cophenetic_reverse divergence,Henon,500,0.785,-0.118,-0.093,-0.094,-0.068,0.799,0.000,0.832,0.000
2,interleaving_0.5_None divergence,Henon,500,0.779,-0.095,-0.092,-0.085,-0.076,0.795,0.000,0.606,0.000
3,interleaving_length_0.5_None divergence,Henon,500,0.786,-0.091,-0.075,-0.086,-0.065,0.825,0.000,0.690,0.000
4,interleaving_edge_0.5_None divergence,Henon,500,0.783,-0.103,-0.086,-0.090,-0.078,0.791,0.000,0.700,0.000


In [26]:
measure_long_name_map = {
    'cophenetic divergence': 'DMT Cophenetic Divergence',
    'cophenetic_reverse divergence': 'DMT Cophenetic (Reverse) Divergence',
    'interleaving_0.5_None divergence': 'PLMT Interleaving (0.5) Divergence',
    'interleaving_0.4_None divergence': 'PLMT Interleaving (0.4) Divergence',
    'interleaving_0.3_None divergence': 'PLMT Interleaving (0.3) Divergence',
    'interleaving_0.2_None divergence': 'PLMT Interleaving (0.2) Divergence',
    'interleaving_0.1_None divergence': 'PLMT Interleaving (0.1) Divergence',
    'interleaving_edge_0.5_None divergence': 'PLMT Interleaving (0.5) Divergence (Edge-Normalised)',
    'interleaving_edge_0.4_None divergence': 'PLMT Interleaving (0.4) Divergence (Edge-Normalised)',
    'interleaving_edge_0.3_None divergence': 'PLMT Interleaving (0.3) Divergence (Edge-Normalised)',
    'interleaving_edge_0.2_None divergence': 'PLMT Interleaving (0.2) Divergence (Edge-Normalised)',
    'interleaving_edge_0.1_None divergence': 'PLMT Interleaving (0.1) Divergence (Edge-Normalised)',
    'interleaving_length_0.5_None divergence': 'PLMT Interleaving (0.5) Divergence (Length-Normalised)',
    'interleaving_length_0.4_None divergence': 'PLMT Interleaving (0.4) Divergence (Length-Normalised)',
    'interleaving_length_0.3_None divergence': 'PLMT Interleaving (0.3) Divergence (Length-Normalised)',
    'interleaving_length_0.2_None divergence': 'PLMT Interleaving (0.2) Divergence (Length-Normalised)',
    'interleaving_length_0.1_None divergence': 'PLMT Interleaving (0.1) Divergence (Length-Normalised)',
    'bottleneck divergence': 'Persistence Bottleneck Divergence',
    'wasserstein divergence': 'Persistence Wasserstein Divergence',
    '$k$-NN graph $M(D)$': 'kNN Graph PD Homology Class Ratio - M(D)',
    '$k$-NN graph $P(D)$': 'kNN Graph PD Maximum Persistence - P(D)',
    '$k$-NN graph $E\'(D)$': 'kNN Graph PD Normalised Persistent Entropy - E\'(D)',
    'Ordinal graph $M(D)$': 'Ordinal Partition Network Graph PD Homology Class Ratio - M(D)',
    'Ordinal graph $P(D)$': 'Ordinal Partition Network Graph PD Maximum Persistence - P(D)',
    'Ordinal graph $E\'(D)$': 'Ordinal Partition Network Graph PD Normalised Persistent Entropy - E\'(D)',
    'Rosenstein': 'Rosenstein Scaling',
    'Eckmann': 'Eckmann Scaling',
    'Kantz': 'Kantz Scaling',
    'cophenetic': 'PLMT Cophenetic Divergence',
    'cophenetic_reverse': 'PLMT Cophenetic (Reverse) Divergence',
    'cophenetic_edge': 'PLMT Cophenetic Divergence (Edge-Normalised)',
    'cophenetic_reverse_edge': 'PLMT Cophenetic (Reverse) Divergence (Edge-Normalised)',
    'cophenetic_length': 'PLMT Cophenetic Divergence (Length-Normalised)',
    'cophenetic_reverse_length': 'PLMT Cophenetic (Reverse) Divergence (Length-Normalised)',
    '$\\Delta$VGA ($L_1$)': 'HVG Top-Bottom Degree Distribution L1 Distance',
    '$\\Delta$VGA ($W_1$)': 'HVG Top-Bottom Degree Distribution Wasserstein Distance',    
    'Betti Vector L1 Norm': 'Persistence Diagram Betti Vector L1 Norm',
}

measure_short_name_map = {
    'cophenetic divergence': 'DMT Cophenetic',
    'cophenetic_reverse divergence': 'DMT Cophenetic (Reverse)',
    'interleaving_0.5_None divergence': 'PLMT Interleaving (0.5)',
    'interleaving_0.4_None divergence': 'PLMT Interleaving (0.4)',
    'interleaving_0.3_None divergence': 'PLMT Interleaving (0.3)',
    'interleaving_0.2_None divergence': 'PLMT Interleaving (0.2)',
    'interleaving_0.1_None divergence': 'PLMT Interleaving (0.1)',
    'interleaving_edge_0.5_None divergence': 'PLMT Interleaving (0.5) / Edges',
    'interleaving_edge_0.4_None divergence': 'PLMT Interleaving (0.4) / Edges',
    'interleaving_edge_0.3_None divergence': 'PLMT Interleaving (0.3) / Edges',
    'interleaving_edge_0.2_None divergence': 'PLMT Interleaving (0.2) / Edges',
    'interleaving_edge_0.1_None divergence': 'PLMT Interleaving (0.1) / Edges',
    'interleaving_length_0.5_None divergence': 'PLMT Interleaving (0.5) / Length',
    'interleaving_length_0.4_None divergence': 'PLMT Interleaving (0.4) / Length',
    'interleaving_length_0.3_None divergence': 'PLMT Interleaving (0.3) / Length',
    'interleaving_length_0.2_None divergence': 'PLMT Interleaving (0.2) / Length',
    'interleaving_length_0.1_None divergence': 'PLMT Interleaving (0.1) / Length',
    'bottleneck divergence': 'PD Bottleneck',
    'wasserstein divergence': 'PD Wasserstein',
    '$k$-NN graph $M(D)$': 'kNN Homology Class Ratio',
    '$k$-NN graph $P(D)$': 'kNN Maximum Persistence',
    '$k$-NN graph $E\'(D)$': 'kNN Persistent Entropy',
    'Ordinal graph $M(D)$': 'OPN Homology Class Ratio',
    'Ordinal graph $P(D)$': 'OPN Maximum Persistence',
    'Ordinal graph $E\'(D)$': 'OPN Persistent Entropy',
    'Rosenstein': 'Rosenstein',
    'Eckmann': 'Eckmann',
    'Kantz': 'Kantz',
    'cophenetic': 'PLMT Cophenetic',
    'cophenetic_reverse': 'PLMT Cophenetic (Reverse)',
    'cophenetic_edge': 'PLMT Cophenetic / Edges',
    'cophenetic_reverse_edge': 'PLMT Cophenetic (Reverse) / Edges',
    'cophenetic_length': 'PLMT Cophenetic / Length',
    'cophenetic_reverse_length': 'PLMT Cophenetic (Reverse) / Length',
    '$\\Delta$VGA ($L_1$)': 'HVG L1',
    '$\\Delta$VGA ($W_1$)': 'HVG W1',
    'Betti Vector L1 Norm': 'Betti L1',
}

measure_type_map = {
    'cophenetic divergence': 'DMT Cophenetic',
    'cophenetic_reverse divergence': 'DMT Cophenetic',
    'interleaving_0.5_None divergence': 'PLMT Interleaving',
    'interleaving_0.4_None divergence': 'PLMT Interleaving',
    'interleaving_0.3_None divergence': 'PLMT Interleaving',
    'interleaving_0.2_None divergence': 'PLMT Interleaving',
    'interleaving_0.1_None divergence': 'PLMT Interleaving',
    'interleaving_edge_0.5_None divergence': 'PLMT Interleaving (e-Normalised)',
    'interleaving_edge_0.4_None divergence': 'PLMT Interleaving (e-Normalised)',
    'interleaving_edge_0.3_None divergence': 'PLMT Interleaving (e-Normalised)',
    'interleaving_edge_0.2_None divergence': 'PLMT Interleaving (e-Normalised)',
    'interleaving_edge_0.1_None divergence': 'PLMT Interleaving (e-Normalised)',
    'interleaving_length_0.5_None divergence': 'PLMT Interleaving (l-Normalised)',
    'interleaving_length_0.4_None divergence': 'PLMT Interleaving (l-Normalised)',
    'interleaving_length_0.3_None divergence': 'PLMT Interleaving (l-Normalised)',
    'interleaving_length_0.2_None divergence': 'PLMT Interleaving (l-Normalised)',
    'interleaving_length_0.1_None divergence': 'PLMT Interleaving (l-Normalised)',
    'bottleneck divergence': 'TDA Divergence',
    'wasserstein divergence': 'TDA Divergence',
    '$k$-NN graph $M(D)$': 'TDA Embedding (kNN)',
    '$k$-NN graph $P(D)$': 'TDA Embedding (kNN)',
    '$k$-NN graph $E\'(D)$': 'TDA Embedding (kNN)',
    'Ordinal graph $M(D)$': 'TDA Embedding (OPN)',
    'Ordinal graph $P(D)$': 'TDA Embedding (OPN)',
    'Ordinal graph $E\'(D)$': 'TDA Embedding (OPN)',
    'Rosenstein': 'Classical Embedding',
    'Eckmann': 'Classical Embedding',
    'Kantz': 'Classical Embedding',
    'cophenetic': 'PLMT Cophenetic',
    'cophenetic_reverse': 'PLMT Cophenetic',
    'cophenetic_edge': 'PLMT Cophenetic (e-Normalised)',
    'cophenetic_reverse_edge': 'PLMT Cophenetic (e-Normalised)',
    'cophenetic_length': 'PLMT Cophenetic (l-Normalised)',
    'cophenetic_reverse_length': 'PLMT Cophenetic (l-Normalised)',
    '$\\Delta$VGA ($L_1$)': 'HVG Graph Distance',
    '$\\Delta$VGA ($W_1$)': 'HVG Graph Distance',
    'Betti Vector L1 Norm': 'TDA Direct',
}

In [27]:
# add measure type column
get_type = lambda row: measure_type_map[row['Measure']]
merged_data['Measure Type'] = merged_data.apply(get_type, axis=1)

In [28]:
# standardise measure names with more useful info
for k, v in measure_short_name_map.items():
    merged_data["Measure"] = merged_data["Measure"].replace(k, v)


In [29]:
merged_data.head()

,Measure,System,Length,Random Forest F1 Score,Random Forest MSE (All),Random Forest MSE (Chaos),kNN MSE (All),kNN MSE (Chaos),Spearman Rho (All),Spearman p (All),Spearman Rho (Chaos),Spearman p (Chaos),Measure Type
0,DMT Cophenetic,Henon,500,0.770,-0.088,-0.065,-0.085,-0.060,0.736,0.000,0.747,0.000,DMT Cophenetic
1,DMT Cophenetic (Reverse),Henon,500,0.785,-0.118,-0.093,-0.094,-0.068,0.799,0.000,0.832,0.000,DMT Cophenetic
2,PLMT Interleaving (0.5),Henon,500,0.779,-0.095,-0.092,-0.085,-0.076,0.795,0.000,0.606,0.000,PLMT Interleaving
3,PLMT Interleaving (0.5) / Length,Henon,500,0.786,-0.091,-0.075,-0.086,-0.065,0.825,0.000,0.690,0.000,PLMT Interleaving (l-Normalised)
4,PLMT Interleaving (0.5) / Edges,Henon,500,0.783,-0.103,-0.086,-0.090,-0.078,0.791,0.000,0.700,0.000,PLMT Interleaving (e-Normalised)


## Regression Performance: Predicting Largest Lyapunov Exponent $\lambda_{\max}$ in the Chaotic Regime

This table reports results of $k$-nearest neighbour regression using $k=10$ neighbours and uniform distance weighting of each neighbour. Values reported in the four system columns are means of (negative) mean squared error (MSE) values over 10 train-test splits of chaotic trajectories of length 500, with the corresponding largest Lyapunov exponent (LLE) as ground truth, from the given systems. Control parameter values generating each system trajectory are sampled uniformly from pre-chosen ranges to ensure inclusion of samples close to bifurcation points. Ground truth LLEs were calculated using Benettin's Jacobian-matrix-based algorithm, on which basis trajectories were then filtered, with those satisfying $\lambda_{\max}>0$ retained. This provided a wide range of representative positive $\lambda_{\max}$ values for each of the systems.

The _Overall Rank_ column gives the overall ranked performance of each measure across all the systems considered.

The best performing feature for prediction of $\lambda_{\max}$ for short-length time series is the _discrete merge tree cophenetic divergence_. This outperforms the well-established and widely used Kantz algorithm, which ranks second, and it achieves lower variance in its respective performance.

In [59]:
df = merged_data
filtered_df = df[df['Length'] == 1000]
knn_mse_pivot_table = pd.pivot_table(filtered_df, values='kNN MSE (Chaos)', index=['Measure Type', 'Measure'], columns=['System'], aggfunc='mean')
rank_columns = []
for col in knn_mse_pivot_table.columns:
    rank_col_name = f"{col} Rank"
    knn_mse_pivot_table[rank_col_name] = knn_mse_pivot_table[col].rank(method='min', ascending=False).astype(int)
    rank_columns.append(rank_col_name)
knn_mse_pivot_table['Mean Rank'] = knn_mse_pivot_table[rank_columns].mean(axis=1)
knn_mse_pivot_table['STD Rank'] = knn_mse_pivot_table[rank_columns].std(axis=1)
knn_mse_pivot_table['Overall Rank'] = knn_mse_pivot_table['Mean Rank'].rank(method='min', ascending=True).astype(int)
pd.options.display.float_format = '{:,.3f}'.format
display(knn_mse_pivot_table)

System                                                               Henon  \
Measure Type                     Measure                                     
Classical Embedding              Eckmann                            -0.044   
                                 Kantz                              -0.022   
                                 Rosenstein                         -0.063   
DMT Cophenetic                   DMT Cophenetic                     -0.072   
                                 DMT Cophenetic (Reverse)           -0.071   
HVG Graph Distance               HVG L1                             -0.115   
                                 HVG W1                             -0.087   
PLMT Cophenetic                  PLMT Cophenetic                    -0.042   
                                 PLMT Cophenetic (Reverse)          -0.042   
PLMT Cophenetic (e-Normalised)   PLMT Cophenetic (Reverse) / Edges  -0.068   
                                 PLMT Cophenetic / Edges            -0.067   
PLMT Cophenetic (l-Normalised)   PLMT Cophenetic (Reverse) / Length -0.081   
                                 PLMT Cophenetic / Length           -0.081   
PLMT Interleaving                PLMT Interleaving (0.1)            -0.062   
                                 PLMT Interleaving (0.2)            -0.071   
                                 PLMT Interleaving (0.3)            -0.055   
                                 PLMT Interleaving (0.4)            -0.112   
                                 PLMT Interleaving (0.5)            -0.090   
PLMT Interleaving (e-Normalised) PLMT Interleaving (0.1) / Edges    -0.042   
                                 PLMT Interleaving (0.2) / Edges    -0.067   
                                 PLMT Interleaving (0.3) / Edges    -0.052   
                                 PLMT Interleaving (0.4) / Edges    -0.099   
                                 PLMT Interleaving (0.5) / Edges    -0.053   
PLMT Interleaving (l-Normalised) PLMT Interleaving (0.1) / Length   -0.060   
                                 PLMT Interleaving (0.2) / Length   -0.078   
                                 PLMT Interleaving (0.3) / Length   -0.062   
                                 PLMT Interleaving (0.4) / Length   -0.082   
                                 PLMT Interleaving (0.5) / Length   -0.053   
TDA Direct                       Betti L1                           -0.071   
TDA Divergence                   PD Bottleneck                      -0.114   
                                 PD Wasserstein                     -0.127   
TDA Embedding (OPN)              OPN Homology Class Ratio           -0.037   
                                 OPN Maximum Persistence            -0.044   
                                 OPN Persistent Entropy             -0.063   
TDA Embedding (kNN)              kNN Homology Class Ratio           -0.048   
                                 kNN Maximum Persistence            -0.085   
                                 kNN Persistent Entropy             -0.089   

System                                                               Ikeda  \
Measure Type                     Measure                                     
Classical Embedding              Eckmann                            -0.097   
                                 Kantz                              -0.122   
                                 Rosenstein                         -0.135   
DMT Cophenetic                   DMT Cophenetic                     -0.050   
                                 DMT Cophenetic (Reverse)           -0.061   
HVG Graph Distance               HVG L1                             -0.102   
                                 HVG W1                             -0.126   
PLMT Cophenetic                  PLMT Cophenetic                    -0.138   
                                 PLMT Cophenetic (Reverse)          -0.132   
PLMT Cophenetic (e-Normalised)   PLMT Cophenetic (Reverse) / Edges  -0.106   
                     

In [31]:
print(knn_mse_pivot_table.to_latex(float_format="%.4f"))

\begin{tabular}{llrrrrrrrrrrr}
\toprule
                    & System &   Henon &   Ikeda &  Logistic &  Tinkerbell &  Henon Rank &  Ikeda Rank &  Logistic Rank &  Tinkerbell Rank &  Mean Rank &  STD Rank &  Overall Rank \\
Measure Type & Measure &         &         &           &             &             &             &                &                  &            &           &               \\
\midrule
Classical Embedding & Eckmann & -0.1654 & -0.0721 &   -0.2270 &     -0.0331 &          37 &           7 &             37 &               29 &    27.5000 &   14.1774 &            34 \\
                    & Kantz & -0.0231 & -0.0950 &   -0.0604 &     -0.0159 &           1 &          18 &              5 &                2 &     6.5000 &    7.8528 &             2 \\
                    & Rosenstein & -0.0967 & -0.1198 &   -0.1589 &     -0.0157 &          31 &          31 &             35 &                1 &    24.5000 &   15.7797 &            29 \\
DMT Cophenetic & DMT Cophenetic & -0.0

/var/folders/7h/nhqr9y0j09b78bvjy9hlwfhw0000gn/T/ipykernel_92681/2991495372.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(knn_mse_pivot_table.to_latex(float_format="%.4f"))


## Statistical Dependence Performance

### Spearman's $\rho$ in the Chaotic Regime

Correlation results for the studied measures appear in the table. Values reported in the system columns are Spearman's $\rho$ coefficient, measuring the rank correlation, or strength of monotonic (but possibly non-linear) dependence, between the studied measures and the actual system $\lambda_{\max}$ value. Corresponding $p$-values are generally of the order of $10^{-20}$ or less whenever $|\rho|>0.5$. Trajectory length is 500.

The most highly correlated measures with $\lambda_{\max}$, for the short-length time series considered, are the _discrete merge tree cophenetic divergence_ and its reversed variant (in which the order of leaves in the superlevel tree is reversed for the divergence calculation). These are both discrete merge tree topological divergences. The next most highly correlated group of measures are those in the piecewise linear merge tree interleaving divergences. Despite showing less monotonic dependence on $\lambda_{\max}$ than the discrete merge tree measures do, these are more strongly correlated with $\lambda_{\max}$ than all other existing measures.

### Correlation across all dynamic regimes

It should be noted that the classical embedding-based estimators of Eckmann, Kantz, and Rosenstein are less highly correlated with $\lambda_{\max}$ in the chaotic regime, for the time series considered, than topological divergences are. However, the former estimators are more highly correlated with $\lambda_{\max}$ when non-chaotic sequences are included. Topological divergences are therefore not a general purpose measure for $\lambda_{\max}$ when the true value of the latter may be negative. A split pipeline of chaos detection (classification) followed by Lyapunov exponent estimation using either topological divergences (when chaos has been identified) or other estimators (when chaos is not present) is likely to be more effective than using one or other type of measure exclusively.


In [32]:
df = merged_data
filtered_df = df[df['Length'] == 500]
spearman_rho_pivot_table = pd.pivot_table(filtered_df, values='Spearman Rho (All)', index=['Measure Type', 'Measure'], columns=['System'], aggfunc='mean')
rank_columns = []
for col in spearman_rho_pivot_table.columns:
    rank_col_name = f"{col} Rank"
    spearman_rho_pivot_table[rank_col_name] = spearman_rho_pivot_table[col].abs().rank(method='min', ascending=False).astype(int)
    rank_columns.append(rank_col_name)
spearman_rho_pivot_table['Mean Rank'] = spearman_rho_pivot_table[rank_columns].mean(axis=1)
spearman_rho_pivot_table['STD Rank'] = spearman_rho_pivot_table[rank_columns].std(axis=1)
spearman_rho_pivot_table['Overall Rank'] = spearman_rho_pivot_table['Mean Rank'].rank(method='min', ascending=True).astype(int)
display(spearman_rho_pivot_table)

System                                                               Henon  \
Measure Type                     Measure                                     
Classical Embedding              Eckmann                             0.437   
                                 Kantz                               0.925   
                                 Rosenstein                          0.900   
DMT Cophenetic                   DMT Cophenetic                      0.736   
                                 DMT Cophenetic (Reverse)            0.799   
HVG Graph Distance               HVG L1                              0.470   
                                 HVG W1                              0.543   
PLMT Cophenetic                  PLMT Cophenetic                     0.829   
                                 PLMT Cophenetic (Reverse)           0.831   
PLMT Cophenetic (e-Normalised)   PLMT Cophenetic (Reverse) / Edges   0.837   
                                 PLMT Cophenetic / Edges             0.835   
PLMT Cophenetic (l-Normalised)   PLMT Cophenetic (Reverse) / Length  0.823   
                                 PLMT Cophenetic / Length            0.820   
PLMT Interleaving                PLMT Interleaving (0.1)             0.729   
                                 PLMT Interleaving (0.2)             0.757   
                                 PLMT Interleaving (0.3)             0.770   
                                 PLMT Interleaving (0.4)             0.769   
                                 PLMT Interleaving (0.5)             0.795   
PLMT Interleaving (e-Normalised) PLMT Interleaving (0.1) / Edges     0.732   
                                 PLMT Interleaving (0.2) / Edges     0.776   
                                 PLMT Interleaving (0.3) / Edges     0.768   
                                 PLMT Interleaving (0.4) / Edges     0.767   
                                 PLMT Interleaving (0.5) / Edges     0.791   
PLMT Interleaving (l-Normalised) PLMT Interleaving (0.1) / Length    0.783   
                                 PLMT Interleaving (0.2) / Length    0.797   
                                 PLMT Interleaving (0.3) / Length    0.788   
                                 PLMT Interleaving (0.4) / Length    0.785   
                                 PLMT Interleaving (0.5) / Length    0.825   
TDA Direct                       Betti L1                           -0.808   
TDA Divergence                   PD Bottleneck                      -0.714   
                                 PD Wasserstein                      0.326   
TDA Embedding (OPN)              OPN Homology Class Ratio            0.815   
                                 OPN Maximum Persistence             0.715   
                                 OPN Persistent Entropy              0.828   
TDA Embedding (kNN)              kNN Homology Class Ratio            0.813   
                                 kNN Maximum Persistence             0.354   
                                 kNN Persistent Entropy              0.318   

System                                                               Ikeda  \
Measure Type                     Measure                                     
Classical Embedding              Eckmann                             0.015   
                                 Kantz                               0.758   
                                 Rosenstein                          0.784   
DMT Cophenetic                   DMT Cophenetic                      0.645   
                                 DMT Cophenetic (Reverse)            0.639   
HVG Graph Distance               HVG L1                              0.429   
                                 HVG W1                              0.447   
PLMT Cophenetic                  PLMT Cophenetic                     0.608   
                                 PLMT Cophenetic (Reverse)           0.560   
PLMT Cophenetic (e-Normalised)   PLMT Cophenetic (Reverse) / Edges   0.571   
                     

## Classification Performance: Predicting the Presence of Chaos

This table reports results of a random forest classification built from $n=100$ decision trees. Values reported in the four system columns are means of the F1 score values values over 10 train-test splits of 500 chaotic and non-chaotic trajectories of length 500, with the sign of the corresponding largest Lyapunov exponent (LLE) as ground truth, from the given systems. Control parameter values generating each system trajectory are sampled uniformly from pre-chosen ranges to ensure inclusion of samples close to bifurcation points. Ground truth LLEs were calculated using Benettin's Jacobian-matrix-based algorithm.

The _Overall Rank_ column gives the overall ranked performance of each measure across all the systems considered.

The best performing feature for prediction of $\lambda_{\max}>0$ for short-length time series is the _discrete merge tree cophenetic divergence_. This outperforms the well-established and widely used Kantz algorithm, which ranks second, and it achieves lower variance in its respective performance.

In [51]:
df = merged_data
filtered_df = df[df['Length'] == 1000]
knn_mse_pivot_table = pd.pivot_table(filtered_df, values='Random Forest F1 Score', index=['Measure Type', 'Measure'], columns=['System'], aggfunc='mean')
rank_columns = []
for col in knn_mse_pivot_table.columns:
    rank_col_name = f"{col} Rank"
    knn_mse_pivot_table[rank_col_name] = knn_mse_pivot_table[col].rank(method='min', ascending=False).astype(int)
    rank_columns.append(rank_col_name)
knn_mse_pivot_table['Mean Rank'] = knn_mse_pivot_table[rank_columns].mean(axis=1)
knn_mse_pivot_table['STD Rank'] = knn_mse_pivot_table[rank_columns].std(axis=1)
knn_mse_pivot_table['Overall Rank'] = knn_mse_pivot_table['Mean Rank'].rank(method='min', ascending=True).astype(int)
pd.options.display.float_format = '{:,.3f}'.format
display(knn_mse_pivot_table)

System                                                               Henon  \
Measure Type                     Measure                                     
Classical Embedding              Eckmann                             0.950   
                                 Kantz                               0.987   
                                 Rosenstein                          0.981   
DMT Cophenetic                   DMT Cophenetic                      0.848   
                                 DMT Cophenetic (Reverse)            0.725   
HVG Graph Distance               HVG L1                              0.660   
                                 HVG W1                              0.551   
PLMT Cophenetic                  PLMT Cophenetic                     0.754   
                                 PLMT Cophenetic (Reverse)           0.785   
PLMT Cophenetic (e-Normalised)   PLMT Cophenetic (Reverse) / Edges   0.751   
                                 PLMT Cophenetic / Edges             0.719   
PLMT Cophenetic (l-Normalised)   PLMT Cophenetic (Reverse) / Length  0.781   
                                 PLMT Cophenetic / Length            0.783   
PLMT Interleaving                PLMT Interleaving (0.1)             0.689   
                                 PLMT Interleaving (0.2)             0.677   
                                 PLMT Interleaving (0.3)             0.689   
                                 PLMT Interleaving (0.4)             0.835   
                                 PLMT Interleaving (0.5)             0.880   
PLMT Interleaving (e-Normalised) PLMT Interleaving (0.1) / Edges     0.621   
                                 PLMT Interleaving (0.2) / Edges     0.781   
                                 PLMT Interleaving (0.3) / Edges     0.707   
                                 PLMT Interleaving (0.4) / Edges     0.792   
                                 PLMT Interleaving (0.5) / Edges     0.802   
PLMT Interleaving (l-Normalised) PLMT Interleaving (0.1) / Length    0.669   
                                 PLMT Interleaving (0.2) / Length    0.737   
                                 PLMT Interleaving (0.3) / Length    0.738   
                                 PLMT Interleaving (0.4) / Length    0.740   
                                 PLMT Interleaving (0.5) / Length    0.752   
TDA Direct                       Betti L1                            0.779   
TDA Divergence                   PD Bottleneck                       0.754   
                                 PD Wasserstein                      0.638   
TDA Embedding (OPN)              OPN Homology Class Ratio            0.872   
                                 OPN Maximum Persistence             0.909   
                                 OPN Persistent Entropy              0.930   
TDA Embedding (kNN)              kNN Homology Class Ratio            0.990   
                                 kNN Maximum Persistence             0.970   
                                 kNN Persistent Entropy              0.968   

System                                                               Ikeda  \
Measure Type                     Measure                                     
Classical Embedding              Eckmann                             0.914   
                                 Kantz                               0.901   
                                 Rosenstein                          0.972   
DMT Cophenetic                   DMT Cophenetic                      0.662   
                                 DMT Cophenetic (Reverse)            0.733   
HVG Graph Distance               HVG L1                              0.841   
                                 HVG W1                              0.828   
PLMT Cophenetic                  PLMT Cophenetic                     0.889   
                                 PLMT Cophenetic (Reverse)           0.847   
PLMT Cophenetic (e-Normalised)   PLMT Cophenetic (Reverse) / Edges   0.876   
                     